In [3]:
import sys
import pathlib

import folium
import pandas as pd
import geopandas as gpd

from datetime import datetime

DIR = pathlib.Path('..')
sys.path.append(str(DIR))

import transportanalyst as ta

DATA_DIR = DIR/'data/'
OUT_DIR = DIR/'output/'

%load_ext autoreload
%autoreload 2

Read the API keys. If you don't have an key you can create one for yourself. At the time I am creating this code, Google map api gives you about $500 free credit to use their API. Here API allows for 25000 free calls for a month. You don't need an API key if you build your own OTP server. You can learn how to build your own OTP server here.

In [25]:
f = open(DATA_DIR/"secrets.txt", "r")
here_key = f.readline().split(': ')[1].rstrip()
google_key = f.readline().split(': ')[1].rstrip()

In [26]:
#Read some random locations to be used for routing
gdf = gpd.read_file(DATA_DIR/'cycle.geojson')

m = folium.Map(location=[-36.850806,174.76456], zoom_start=12)
folium.GeoJson(gdf).add_to(m)
m

In [27]:
#Use google api to find the best route best route.
google_route = ta.google_route(
    in_gdf = gdf, 
    trip_name = 'test',
    mode = 'cycle',
    api_key=google_key,
)
folium.GeoJson(google_route).add_to(m)
m

In [28]:
#Use here api to find the best route best route.
here_route = ta.here_route(
    in_gdf = gdf, 
    trip_name = 'test',
    mode = 'public_transport',
    api_key=here_key,
)
here_route

{'mode': 'fastest;publicTransport', 'departure': '2020-08-07T12:38:00', 'apiKey': 'gnxn1zfVqr3j-MhK5unP8ezHcBvbbEeYeNnNt2TMC68', 'waypoint0': '-36.86671222190732,174.74544525146484', 'waypoint1': '-36.81917959744267,174.6390151977539', 'waypoint2': '-36.896645347849706,174.82131958007812'}


{'response': {'metaInfo': {'timestamp': '2020-08-07T00:38:50Z',
   'mapVersion': '8.30.111.154',
   'moduleVersion': '7.2.202031-7544',
   'interfaceVersion': '2.6.76',
   'availableMapVersion': ['8.30.111.154']},
  'route': [{'waypoint': [{'linkId': '-865017866',
      'mappedPosition': {'latitude': -36.8666151, 'longitude': 174.7453927},
      'originalPosition': {'latitude': -36.8667123, 'longitude': 174.7454453},
      'type': 'stopOver',
      'spot': 0.1818182,
      'sideOfStreet': 'left',
      'mappedRoadName': 'Home St',
      'label': 'Home St',
      'shapeIndex': 0,
      'source': 'user'},
     {'linkId': '+737064525',
      'mappedPosition': {'latitude': -36.819005, 'longitude': 174.6394551},
      'originalPosition': {'latitude': -36.8191796, 'longitude': 174.6390152},
      'type': 'stopOver',
      'spot': 0.4460432,
      'sideOfStreet': 'left',
      'mappedRoadName': 'Raphael Pl',
      'label': 'Raphael Pl',
      'shapeIndex': 308,
      'source': 'user'},
     {

In [32]:
#otp doesn't have the waypoints now.
otp_route = ta.otp_route(
    in_gdf = gdf, 
    trip_name = 'test',
    date_time = datetime.strptime('20200109 0800', '%Y%m%d %H%M'),
    mode = 'public_transport',
)
folium.GeoJson(otp_route[['geometry']]).add_to(m)
m

In [35]:
otp_route = ta.otp_service_area(
    in_gdf = gdf,
    mode = 'public_transport', 
    breaks = [10, 20],
    date_time = datetime.strptime('20200109 0800', '%Y%m%d %H%M'),
)
folium.GeoJson(otp_route[['geometry']]).add_to(m)
m

In [36]:
#Read census data to be used for accessibility
gdf = gpd.read_file(DATA_DIR/'census_sa1.geojson')

m = folium.Map(location=[-36.850806,174.76456], zoom_start=14)
folium.GeoJson(gdf).add_to(m)
m

In [37]:
census = ta.otp_accessibility(
    census = gdf,
    fields = ['pow2018', 'pop2018'],
    mode = 'walk',
    travel_time = 5,
)

9.9% is done.
19.9% is done.
29.8% is done.
30.5% is done.
39.7% is done.
40.4% is done.
49.7% is done.
50.3% is done.
59.6% is done.
60.3% is done.
69.5% is done.
70.2% is done.
80.1% is done.
90.1% is done.
100.0% is done.


In [38]:
census.head(2).T

,0,1
SA12018_V1,7005275,7005339
pow2018,18,12
pop2018,78,135
geometry,(POLYGON ((174.7610941835585 -36.8473837666208...,(POLYGON ((174.7626098002672 -36.8566746831506...
centroid,POINT (174.7613105893869 -36.84766302466824),POINT (174.7625749792188 -36.85692751176538)
walk_20200807_1241_pow2018,4594.76,319.87
walk_20200807_1241_pop2018,2278.52,371.764
